# Postprocessing Docking Results

In [1]:
import os, shutil
import pandas as pd

## Turn off SettingWithCopyWarning
pd.set_option('mode.chained_assignment', None)

## Parse AlphaFold2 Results

In [2]:
## Define path to results directory
results_directory_path = "./best_AF2_and_GDock"

In [10]:
## Create empty DataFrame for metrics
experiment_results_df = pd.DataFrame(columns=[
    'experiment_name', 
    'n_protein',
    'cytokine_protein',
    'best_pdb_path',
    'prodigy_deltaG_kcalpermol',
    'prodigy_dissociation_constant_M'
])


## Show DataFrame
experiment_results_df

,experiment_name,n_protein,cytokine_protein,best_pdb_path,prodigy_deltaG_kcalpermol,prodigy_dissociation_constant_M


## Generate PRODIGY Predictions

Note: You must have PRODIGY installed.
```sh
git clone https://github.com/haddocking/prodigy
pip install prodigy/.
```

In [ ]:
## To run on remote SuperCloud (Linux)
local_base_path = "/home/gridsan/cford/seqer_shared"

for pdb_file in os.listdir(results_directory_path):
    experiment_name = pdb_file.split("_relaxed")[0]
    n_protein = experiment_name.split("_")[0].replace("-N", "")
    cytokine_protein = experiment_name.split("_")[1]
    best_pdb_path = f'{results_directory_path}/{pdb_file}'
    best_pdb_path_full = f'{local_base_path}/{results_directory_path.replace("./", "")}/{pdb_file}'
    
    ## Run PRODIGY and parse stdout
    prodigy_output = os.popen(f'/home/gridsan/cford/.local/bin/prodigy {best_pdb_path_full}').read()
    prodigy_output_lines = prodigy_output.split('\n')
    if len(prodigy_output_lines) > 1:
        prodigy_deltaG_kcalpermol = float(prodigy_output_lines[-3].split(':')[1].replace(' ', ''))
        prodigy_dissociation_constant_M = float(prodigy_output_lines[-2].split(':')[1].replace(' ', ''))
        # print(predicted_binding_affinity)

        ## Add values to DataFrame
        experiment_results_df.loc[len(experiment_results_df.index)] = [
                experiment_name,
                n_protein,
                cytokine_protein,
                best_pdb_path,
                prodigy_deltaG_kcalpermol,
                prodigy_dissociation_constant_M] 

experiment_results_df

In [ ]:
## Write out results DataFrame
experiment_results_df.to_csv("best_AF2_and_GDock_experiment_results.csv")